# Just copying some gaussian processes for dummies code to try and figure out what i am doing wrong

In [2]:
%%capture
%pip install numpy
%pip install matplotlib

In [27]:
import numpy as np
import matplotlib.pyplot as plt
import math

def gauss_corr(l, a_vec, b_vec):
    # a and b are points they could be scalar or vector
    exponent = 0

    # i actually don't think i should be adding the exponent
    for a,b in zip(a_vec, b_vec):
        exponent += -(1/2*l)*(a-b)**2
        
    return (math.e)**exponent

#this sends of x1x1 x1x2 and so on to the guass_corr calc
# for 1 input x this isn't all that important but higher dims
# will benifit from this expected form [[x1,x2],[x1,x2]]
def gauss_corr_handler(X1,X2, l=0.1):
    R = []
    X1_shape = (X1.shape)[0]
    X2_shape = (X2.shape)[0]
    R = np.zeros((X1_shape,X2_shape))
    for i, x_vec_1 in enumerate(X1):
        for j, x_vec_2 in enumerate(X2):
            R[i][j] = gauss_corr(l,x_vec_1,x_vec_2)
    return R

n =35

# Noiseless training data
Xtrain = (np.random.rand(n)*2*(math.pi)).reshape(-1,1)
ytrain = np.sin(Xtrain)
Xtest = np.arange(0,2*np.pi,0.1).reshape(-1,1)[:35]
K_xx = gauss_corr_handler(Xtrain, Xtrain, l=0.1)

# Apply the kernel function to our training points
s_d = 0.01
covar_xx = np.linalg.cholesky(K_xx + s_d*np.identity(K_xx.shape[1]))

# Compute the mean at our test points.
#(were k_xs is K_xx' (s as in x star, so test points))
K_xs = gauss_corr_handler(Xtrain, Xtest, l=0.01)
Lk = np.linalg.solve(covar_xx, K_xs)
mu = np.dot(Lk.T, np.linalg.solve(covar_xx, ytrain)).reshape((Xtest.shape[0],))


# Compute the standard deviation so we can plot it
s2 = np.diag(K_xx) - np.sum(Lk**2, axis=0)
stdv = np.sqrt(s2)
# Draw samples from the posterior at our test points.
covar_xx = np.linalg.cholesky(K_xx + 1e-6*np.eye(n) - np.dot(Lk.T, Lk))
f_post = mu.reshape(-1,1) + np.dot(covar_xx, np.random.normal(size=(n,3)))

plt.plot(Xtrain, ytrain, 'bs', ms=8)
plt.plot(Xtest, f_post)
plt.gca().fill_between(Xtest.flat, mu-2*stdv, mu+2*stdv, color="#dddddd")
plt.plot(Xtest, mu, 'r--', lw=2)
plt.axis([-5, 5, -3, 3])
plt.title('Three samples from the GP posterior')
plt.show()

C:\Users\seanp\AppData\Local\Temp\ipykernel_3016\4291062105.py:49: RuntimeWarning: invalid value encountered in sqrt
  stdv = np.sqrt(s2)


LinAlgError: Matrix is not positive definite